# Pyspark의 ALS 활용하기
### https://techblog-history-younghunjo1.tistory.com/161 참고

In [ ]:
!apt-get install openjdk-8-jdk-headless

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz

In [ ]:
!tar -xvf spark-3.2.4-bin-hadoop3.2.tgz

In [ ]:
!pip install findspark

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
import sklearn
import random

# Pysparkk Library #
# SQL
from pyspark import SparkConf, SparkContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
from pyspark.sql.types import *

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [ ]:
# 스파크 세션 만들기
spark = SparkSession\
        .builder\
        .appName('TOPIK Mate ALS')\
        .getOrCreate()

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
# 사용자 데이터
users = pd.read_csv('/content/drive/MyDrive/캡스톤디자인_공유폴더/datas/contents/users_sample.csv')

In [ ]:
users.sample(20)

In [ ]:
# 문제 데이터
problems = pd.read_csv('/content/drive/MyDrive/캡스톤디자인_공유폴더/datas/contents/questions.csv')
problems.sample(20)

In [ ]:
# 답안과 합치기 - pandas.merge()

solved = pd.merge(users, problems)
solved.head()

In [ ]:
del users
del problems

In [ ]:
solved['elapsed_time'] = pd.to_numeric(solved['elapsed_time'])

In [ ]:
solved.sample(20)

In [ ]:
# 문제 ID 별 분류
questions = solved.groupby(solved.question_id)
questions.size()

In [ ]:
del solved

In [ ]:
result_df = pd.DataFrame()

# 그룹별 정/오답 그룹 정규 분포화
idx = 0
for key, group in questions:

    # if (idx == 1):
    #     break;

    # print(f'[{key}] ============= ')

    # group = questions.get_group(key)

    # 정답 그룹
    correct_group = group[group['user_answer'] == group['correct_answer']]
    # 오답 그룹
    wrong_group = group[group['user_answer'] != group['correct_answer']]

    # print(f'group.shape: {group.shape}, correct_group.shape: {correct_group.shape}, wrong_group.shape: {wrong_group.shape}')

    pivot = 0
    # 정답 그룹 pivot 값
    if correct_group.size > 0:

        # N% 구간의 기준값 찾기
        pivot = np.percentile(correct_group['elapsed_time'], 15)
        try:
            correct_group.loc[correct_group['elapsed_time'] <= pivot, 'label'] = 1
            correct_group.loc[correct_group['elapsed_time'] > pivot, 'label'] = 2
        except ValueError as e:
            print('error: ', e)

        result_df = pd.concat([result_df, correct_group], axis=0)

    # print('correct_group: ', correct_group)

    # labeled1 = correct_group
    # labeled1['label'] = np.where(labeled1['elapsed_time'] <= pivot, 1, 2)


    pivot2 = 0
    # 오답 그룹 pivot 값
    if wrong_group.size > 0:
        pivot2 = np.percentile(wrong_group['elapsed_time'], 15)
        try:
            wrong_group.loc[wrong_group['elapsed_time'] <= pivot2, 'label'] = 4
            wrong_group.loc[wrong_group['elapsed_time'] > pivot2, 'label'] = 5
        except ValueError as e:
            print('error: ', e)
        result_df = pd.concat([result_df, wrong_group], axis=0)


    # print('wrong_group: ', wrong_group)
    # labeled2 = wrong_group
    # labeled2['label'] = np.where(labeled2['elapsed_time'] <= pivot2, 5, 4)


    # merge
    # temp = pd.concat([labeled1, labeled2], axis=0)
    # temp = pd.concat([correct_group, wrong_group], axis=0)

    idx += 1


In [ ]:
del questions

In [ ]:
result_df[result_df['question_id'] != 'q1'].sample(20)

In [ ]:
# ALS 추천 알고리즘
from pyspark.ml.recommendation import ALS

In [ ]:
# 스키마 정의
df_schema = StructType([StructField("user_id", StringType(), True)\
                        ,StructField("solving_id", StringType(), True)\
                        ,StructField("question_id", StringType(), True)\
                        ,StructField("user_answer", StringType(), True)\
                        ,StructField("bundle_id", StringType(), True)\
                        ,StructField("explanation_id", StringType(), True)\
                        ,StructField("correct_answer", StringType(), True)\
                        ,StructField("part", StringType(), True)\
                        ,StructField("tags", StringType(), True)\
                        ,StructField("deployed_at", StringType(), True)\
                        ,StructField("elapsed_time", LongType(), True)\
                        ,StructField("label", FloatType(), True)])

# Pandas -> Spark 변환
spark_df = spark.createDataFrame(result_df, schema=df_schema)
display(spark_df)

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString

# 문자열인 user_id, question_id를 수치형 데이터로 바꾸기
stringIndexer = StringIndexer(inputCols=['user_id', 'question_id'],
                              outputCols=['user_id_num', 'question_id_num'],
                              handleInvalid='error',
                              stringOrderType='alphabetDesc')
# stringIndexer = StringIndexer(inputCol='user_id', outputCol='user_id_num')
stringIndexerModel = stringIndexer.fit(spark_df)
encoded_df = stringIndexerModel.transform(spark_df)


In [ ]:
encoded_df.limit(5).toPandas()

In [ ]:
# 학습, 테스트 데이터 분리
train, test = encoded_df.randomSplit([0.75, 0.25])

In [ ]:
rec = ALS(maxIter = 10,
          regParam = 0.01,
          userCol = 'user_id_num',
          itemCol = 'question_id_num',
          ratingCol = 'label',
          nonnegative = True,
          coldStartStrategy='drop')


# ALS 모델 학습 -> dataframe을 넣어주기
rec_model = rec.fit(train)

In [ ]:
# trainsform을 이용해 예측 -> dataframe을 넣어주기
pred_labels = rec_model.transform(test)
pred_labels.limit(5)

In [ ]:
# 실제 평점과 예측 평점 사이의 차이값으로 RMSE와 MAE값을 측정
# Get metric for training
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol='label',
                                predictionCol='prediction',
                                metricName='rmse')

# evaluate 메소드에 예측값 담겨있는 dataframe 넣어주기
rmse = evaluator.evaluate(pred_labels)

mae_eval = RegressionEvaluator(labelCol='label',
                               predictionCol='prediction',
                               metricName='mae')

mae = mae_eval.evaluate(pred_labels)

print('RMSE: ', rmse)
print('MAE: ', mae)

In [ ]:
encoded_df.head()

# 특정 유저에게 문제 추천해주기

In [ ]:
import sklearn
import random
import os
import pandas as pd
import numpy as np

In [ ]:
# Pyspark Library #
# SQL
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
# ML
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
unique_questions = encoded_df.select('question_id_num').distinct()
encoded_df.toPandas()

In [ ]:
def top_questions(user_id, n):
    """
    특정 유저에게 도움이 될 만한 n개의 문제를 추천
    """

    # unique_questions 를 a라고 별칭
    a = unique_questions.alias('a')

    # 특정 유저가 푼 문제들만 담은 새로운 데이터프레임 생성
    solved_questions = encoded_df.filter(encoded_df['user_id'] == user_id)\
                                    .select('question_id_num')

    # 특정 유저가 푼 문제들을 b라고 별칭
    b = solved_questions.alias('b')

    # unique_questions를 기준으로 solved_questions를 조인시켜서
    # 유저가 풀지 않은 문제들 파악 가능
    total_questions = a.join(b, a['question_id_num'] == b['question_id_num'],
                             how='left')

    # b 데이터프레임의 question_id_num값이 결측치를 갖고 있는 행의
    # a.question_id_num를 뽑아냄으로써 유저가 아직 보지 못한 문제들을 추출
    remaining_questionis = total_questions\
                            .where(col('b.question_id_num').isNull())\
                            .select('a.question_id_num').distinct()

    print(remaining_questionis.limit(5).toPandas())

    # remaining_questions 데이터프레임에 특정 유저값을 동일하게 새로운 변수로 추가해주기
    # remaining_questions = remaining_questions.withColumn('user_id', lit(int(user_id)))

    # 위에서 만든 ALS 모델을 사용하여 추천 평점 예측 후 n개 만큼 view ->
    recommender = rec_model.transform(remaining_questionis)\
                            .orderBy('prediction', ascending=False)\
                            .limit(n)

    print(recommender)

    # StringIndexer로 만든 것을 역으로
    question_ids = IndexToString(inputCols = ['user_id_num', 'question_id_num'],
                                 outputCols = ['user_id', 'question_id'],
                                 labels=model.label)

    # transform해서 문제id, 유저id를 숫자 -> 문자열로 변환
    final_recommendations = question_ids.transform(recommender)

    return final_recommendations.show(n, truncate=False)


In [ ]:
top_questions('u1000', 5)